In [2]:

from rouge import Rouge
import numpy as np
from scipy.stats import pearsonr

In [3]:
meaning={6:3,7:3,9:3,11:2,12:2,14:2,21:1,22:1,24:1}
grammer={6:3,7:2,9:1,11:3,12:2,14:1,21:3,22:2,24:1}

In [4]:
def ReadTsv(path):
    data=[]
    with open(path,"r") as f:
        count=0
        for lines in f:
            temp=[]
            x=lines.replace('|||','\t\t')
            x= x.split('\t')
            text = x[2]
            temp.append(text[0:-1])
            temp.append(x[4])
            c=0
            n=0
            k=0
            gram_sum =0
            fluen_sum=0
            score=0
            
            for i,y in enumerate(x):
                if (k!=0):
                    k-=1
                    continue
                if i<6:
                    continue
                if c==0:
                    n = int(y)
                    c=1
                    continue
                
                if (not(y) or i==len(x)-2):
                    temp.append(str(gram_sum/n))
                    temp.append(str(fluen_sum/n))
                    temp.append(str(score/n))
                    fluen_sum=0
                    gram_sum=0
                    data.append(temp)
                    temp=[text,x[i+1]]
                    k=2
                    c=0
                    continue

                gram_sum+=grammer[int(y)]
                fluen_sum+=meaning[int(y)]
                score+=int(y)
                k=1
            count+=1
                
    return (data)  
train =ReadTsv('./RawData/train.tsv')      
valid = ReadTsv('./RawData/valid.tsv')
test = ReadTsv('./RawData/test.tsv')



3370


In [28]:
# write train into train.tsv file
with open('./RawData/temp.tsv','w') as f:
    for i in test:
        f.write(i[0]+'|||'+i[1]+'|||'+i[2]+'|||'+i[3]+'|||'+i[4]+'\n')

print(len(test))


3370


In [6]:
def Rogue_L(compression,sentence):
        rouge =Rouge()
        scores = rouge.get_scores(compression, sentence)
        return (scores[0]['rouge-l']['f'])
def Rogue_1(compression,sentence):
        rouge =Rouge()
        scores = rouge.get_scores(compression, sentence)
        return (scores[0]['rouge-1']['f'])
def Rogue_2(compression,sentence):
        rouge =Rouge()
        scores = rouge.get_scores(compression, sentence)
        return (scores[0]['rouge-2']['f'])

In [7]:


MSE_ROUGEL=[]
for i in range(len(train)): 
    x=Rogue_L(train[i][0],train[i][1])
    MSE_ROUGEL.append(x)


In [8]:
MSE_ROUGE1=[]
for i in range(len(train)): 
    x=Rogue_1(train[i][0],train[i][1])
    MSE_ROUGE1.append(x)

In [9]:
MSE_ROUGE2=[]
for i in range(len(train)): 
    x=Rogue_2(train[i][0],train[i][1])
    MSE_ROUGE2.append(x)


In [10]:
# convert all of them into numpy arrays

MSE_ROUGEL = np.array(MSE_ROUGEL)
MSE_ROUGE1 = np.array(MSE_ROUGE1)
MSE_ROUGE2 = np.array(MSE_ROUGE2)

In [11]:
grammer_mat=[]
meaning_mat=[]
score_mat=[]
for i in range(len(train)):
    grammer_mat.append(float(train[i][2]))
    meaning_mat.append(float(train[i][3]))
    score_mat.append(float(train[i][4]))

grammer_mat = np.array(grammer_mat)
meaning_mat = np.array(meaning_mat)
score_mat = np.array(score_mat)

In [12]:
correlation, _ = pearsonr(grammer_mat, MSE_ROUGEL)
print('Pearsons correlation between grammer and ROUGE-L: %.3f' % correlation)

Pearsons correlation between grammer and ROUGE-L: 0.102


In [13]:
correlation, _ = pearsonr(meaning_mat, MSE_ROUGEL)
print('Pearsons correlation between meaning_mat and ROUGE-L: %.3f' % correlation)

Pearsons correlation between meaning_mat and ROUGE-L: 0.113


In [14]:
correlation, _ = pearsonr(score_mat, MSE_ROUGEL)
print('Pearsons correlation between score_mat and ROUGE-L: %.3f' % correlation)

Pearsons correlation between score_mat and ROUGE-L: -0.118


In [15]:
correlation, _ = pearsonr(grammer_mat, MSE_ROUGE1)
print('Pearsons correlation between grammer and ROUGE-1: %.3f' % correlation)

Pearsons correlation between grammer and ROUGE-1: 0.120


In [16]:
correlation, _ = pearsonr(meaning_mat, MSE_ROUGE1)
print('Pearsons correlation between meaning_mat and ROUGE-1: %.3f' % correlation)

Pearsons correlation between meaning_mat and ROUGE-1: 0.131


In [17]:
correlation, _ = pearsonr(score_mat, MSE_ROUGE1)
print('Pearsons correlation between score_mat and ROUGE-1: %.3f' % correlation)

Pearsons correlation between score_mat and ROUGE-1: -0.136


In [18]:
# get the last coloumn of temp.tsv for each line where each are seperated by tab delimiters
wpslor=[]
with open('./RawData/WPSLOR.tsv','r') as f:
    for lines in f:
        x=lines.replace('|||','\t\t')
        # print(x)
        x=x.split('\t')
        wpslor.append(float(x[-1]))

wpslor = np.array(wpslor)
def skip_correlation(y1,y2):
    mask = np.logical_and(np.isfinite(y1), np.isfinite(y2))
    filtered_x = y1[mask]
    filtered_y = y2[mask]
    correlation, _ = pearsonr(filtered_x, filtered_y)
    
    return correlation


# Calculate Pearson correlation coefficient
correlation= skip_correlation(wpslor, grammer_mat)
    
print('Pearsons correlation between wpslor and grammer_mat: %.3f' % correlation)

correlation= skip_correlation(wpslor, meaning_mat)
    
print('Pearsons correlation between wpslor and meaning_mat: %.3f' % correlation)

correlation= skip_correlation(wpslor, score_mat)
    
print('Pearsons correlation between wpslor and score_mat: %.3f' % correlation)




Pearsons correlation between wpslor and grammer_mat: 0.069
Pearsons correlation between wpslor and meaning_mat: 0.020
Pearsons correlation between wpslor and score_mat: -0.017


In [31]:
grammer_mat_test=[]
meaning_mat_test=[]
for i in range(len(test)):
    grammer_mat_test.append(float(test[i][2]))
    meaning_mat_test.append(float(test[i][3]))

grammer_mat_test = np.array(grammer_mat_test)
meaning_mat_test = np.array(meaning_mat_test)

In [32]:
slor_mat=np.load('./RawData/slor_1.npy')

correlation = skip_correlation(slor_mat, grammer_mat_test)
print('Pearsons correlation between slor_mat and grammer_mat: %.3f' % correlation)
correlation = skip_correlation(slor_mat, meaning_mat_test)
print('Pearsons correlation between slor_mat and meaning_mat_test: %.3f' % correlation)

Pearsons correlation between slor_mat and grammer_mat: 0.102
Pearsons correlation between slor_mat and meaning_mat_test: 0.094
